In [1]:
import numpy as np

In [ ]:
def gym(players):
    n = len(self.players)
    base_coin = 21 - np.clip(n, 5, 7)*2
    replay = []
    terminal = []
  
    def init():
        deck = np.arange(33) + 3
        np.random.shuffle(deck)
        deck = list(deck[:24])
        state = [[base_coin, []] for _ in range(n)]
        turn = np.random.randint(n)
        pot = None
        return state, turn, pot, deck
    
    def get_score(hand):
        out = hand[0]
        for p, q in zip(hand[:-1], hand[1:])
            if p + 1 != q:
                out += q
        return out
    
    def get_serial(state, pot): # 0~55 (56) + 3~35 (33)
        out = np.zeros((n+1, 89), int)
        for i in range(n):
            out[i,state[i][0]] = 1
            for c in state[i][1]:
                out[i,c+53] = 1
        if pot is not None:
            out[-1,pot[0]] = 1
            out[-1,pot[1]+53] = 1
        return out
    
    def run():
        state, turn, pot, deck = init()
        
        def play():
            nothanks = state[turn][0] > 0 && players[turn](state[turn:] + state[:turn] + pot)
            s0 = get_serial(state, pot)
            if nothanks:
                state[turn,0] -= 1
                pot[0] += 1
                s1 = get_serial(state, pot)
                replay.append(-1, s0, s1)
            else:
                state[turn,0] += pot[0]
                state[turn,1].append(pot[1])
                state[turn,1].sort()
                pot = None
                s1 = get_serial(state, pot)
                replay.append(s0, s1)
            return nothanks
        
        while len(deck) > 0:
            pot = [0, deck.pop()]
            
        
        
        
        
    return run 

In [65]:
np.zeros((2+1, 89), int).dtype

dtype('int32')

In [30]:
class Gym:
    def __init__(self, players):
        n = len(players)
        assert 3 <= n and n <= 7
        self.players = players
        self.record = []
        self.terminal = []
    
    def clear(self):
        n = len(self.players)
        base_coin = 21 - np.clip(n, 5, 7)*2
        self.state = [[base_coin, []] for _ in range(n)]
        self.pot = None
        self.turn = np.random.randint(n)
    
    def shuffle(self):
        cards = np.arange(33) + 3
        np.random.shuffle(cards)
        this.deck = list(cards[:24])
    
    def get_score(self, state):
        out = state[0,0]
        if state[0,1] == 1: 
            out -= 3
        for i in range(2, 34):
            if state[0,i] == 1 and state[0,i-1] == 0:
                out -= i + 2
        return out
    
    def get_serial_(self, coin, hand):
        return np.r_[[coin] + list(np.r_[[(card == np.arange(33) + 2).astype(int) for card in hand]].sum(axis=0))]
    
    def get_serial(self):
        return np.r_[[self.get_serial_(*s) for s in (self.state[self.turn:] + self.state[:self.turn])] + [self.get_serial_(*self.pot)]]
    
    def play(self):
        if self.pot is None
            if len(self.deck) == 0: return False
            self.pot = [0, [self.deck.pop()]]
        else:
            nothanks = (self.state[self.turn][0] > 0) && self.players[self.turn](self.state[self.turn:] + self.state[:self.turn], self.pot)
            if nothanks:
                s0 = 
                self.state[turn,0] -= 1
                self.pot[0] += 1
                print(s0.shape)
                
                replay = [self.state[turn:] + self.state[:turn], self.pot]
                self.state[turn,0] -= 1
                self.pot[0] += 1
                replay.append(np.r_[self.state, self.pot])
            else:
                
                
            
        return True
            
        

SyntaxError: invalid syntax (<ipython-input-30-5311dc1acf83>, line 31)

In [35]:
np.r_[1,2,3]

array([1, 2, 3], dtype=int32)

In [38]:
np.r_[[0] + (np.arange(33) == 12)]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
hand = [5,6,7]
print(list(np.r_[[(card == np.arange(33) + 2).astype(int) for card in hand]].sum(axis=0)))

[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [50]:
coin = 10
hand = [25,26,17]
np.r_[[coin] + list(np.r_[[(card == np.arange(33) + 2).astype(int) for card in hand]].sum(axis=0))]

array([10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0])

In [51]:
state = [np.r_[[coin] + list(np.r_[[(card == np.arange(33) + 2).astype(int) for card in hand]].sum(axis=0))] for _ in range(5)]

In [53]:
state = [[1, [3, 4]], [2, [5, 6]], [7, [8, 9]], [10, [11, 12]], [13, [14, 15]]]

In [54]:
def get_serial(coin, hand):
        return np.r_[[coin] + list(np.r_[[(card == np.arange(33) + 2).astype(int) for card in hand]].sum(axis=0))]

In [61]:
np.r_[[get_serial(*s) for s in state] + [get_serial(*[2,[20]])]]

array([[ 1,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 2,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 7,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0],
       [ 2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]])